## Wir haben zwei Spark installiert.

## Auf Windows und WSL Ubuntu 22.04 mit Hadoop/Spark


### Sessionaufbau für Spark-Windows

In [1]:
# Import SparkSession
from pyspark.sql import SparkSession

# Create SparkSession 
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("MeinProgramm.com") \
      .getOrCreate() 

type(spark)

pyspark.sql.session.SparkSession

In [5]:
spark.stop()

In [ ]:
## Beispiel einlesen (read einer CSV, die unter Windows gespeichert ist, im selben Ordner wie dieses Notebook)

Sdf1 = spark.read.csv("all_us_zipcodes.csv")

print(Sdf1.printSchema())
print(Sdf1.show())

# oder

Sdf2 = spark.read.format('csv').load("zipcode.csv")
print(Sdf2.printSchema())
print(Sdf2.show())

print(type(Sdf2))


### Schreiben eines Spark DataFrame in eine Datei unter Windows

In [ ]:
# Importiere die notwendigen Bibliotheken
import pandas as pd

# Konvertiere den Spark DataFrame in einen Pandas DataFrame
pandas_df = Sdf1.toPandas()

# Speichere den Pandas DataFrame als eine einzelne CSV-Datei
output_path = "test.csv"  # im gleichen Ordner wie das Notebook
# oder mit Pfadangabe
# output_path = "c:\\0_BigData\\test.csv"

pandas_df.to_csv(output_path, index=False)

spark.stop()


In [ ]:
'''
Der Code liefert Fehler:

import os
from pyspark.sql import SparkSession

# Create SparkSession 
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("MeinProgramm.com") \
      .getOrCreate() 

# Lies den Spark DataFrame ein
sdf1 = spark.read.csv("all_us_zipcodes.csv", header=True, inferSchema=True)

# Definiere den Ausgabeordner
output_folder = "c:\\0_BigData\\Source"

# Stelle sicher, dass der Ausgabeordner existiert
os.makedirs(output_folder, exist_ok=True)

# Schreibe den Spark DataFrame in Partitionen
partitioned_path = os.path.join(output_folder, "partitioned_data")

sdf1.repartition(2).write.csv(partitioned_path, header=True, mode="overwrite")

# Generator, um die Partitionen zu lesen und zu einer einzigen Datei zusammenzuführen
def merge_partitions_to_single_csv(partitioned_path, output_file):
    # Suche alle Partitionen
    partition_files = [os.path.join(partitioned_path, f) for f in os.listdir(partitioned_path) if f.startswith("part-")]
    
    # Schreibe die Daten von allen Partitionen in eine einzelne Datei
    with open(output_file, 'w', encoding='utf-8') as outfile:
        for i, partition_file in enumerate(partition_files):
            with open(partition_file, 'r', encoding='utf-8') as infile:
                for line in infile:
                    if i != 0 and infile.tell() == 0:
                        # Überspringe die Headerzeile nach der ersten Partition
                        continue
                    outfile.write(line)

# Definiere den endgültigen Dateinamen
final_output_file = os.path.join(output_folder, "final_output.csv")

# Führe die Partitionen zusammen
merge_partitions_to_single_csv(partitioned_path, final_output_file)
'''


# Sessionaufbau für Spark-Hadoop auf Ubuntu

In [ ]:
pip install grpcio

In [ ]:
pip install grpcio-status --use-deprecated=legacy-resolver

In [ ]:
pip install google-api-core --use-deprecated=legacy-resolver

In [1]:
# Jetzt kann eine Session mit Spark auf Hadoop erstellt werden
# Hierzu benötigen wir wieder die IP-Adresse des WSL-Ubuntu
# Zu finden auf Ubuntu mit  hostname -I

from pyspark.sql import SparkSession

spark = SparkSession.builder.remote("sc://172.21.82.235:15002") \
                            .appName("MeinProgramm.com") \
                            .getOrCreate()
type(spark)

pyspark.sql.connect.session.SparkSession

In [ ]:
import time

time.sleep(10)  # Wait for 10 seconds

spark = SparkSession.builder.remote("sc://172.21.82.235:15002") \
                    .appName("MeinProgramm.com") \
                    .getOrCreate()


In [ ]:
spark.stop()

In [ ]:
## Beispiel einlesen (read einer CSV, die im HDFS gespeichert ist)

# Sdf = spark.read.csv("all_us_zipcodes.csv")
# Problem: So erwartet er die Datei nicht in der hdfs-Root, sondern unter /user/alfa

# Lies die Datei aus einem beliebigen Ordner im HDFS
hdfs_path = "hdfs://172.26.195.52:9000/bigdata/test123.csv"
Sdf = spark.read.csv(hdfs_path)

print(Sdf.printSchema())
print(Sdf.show())




In [ ]:
# Schreiben eines Spark DataFrame in Hadoop hdfs

output_path = "/bigdata"

Sdf.write.mode("overwrite").csv(output_path, header=True)


In [ ]:
from pyspark.sql import SparkSession
from hdfs import InsecureClient

# Remote SparkSession erstellen
spark = SparkSession.builder.remote("sc://172.26.195.52:15002") \
                    .appName("MeinProgramm.com") \
                    .getOrCreate()

Sdf = spark.read.csv("all_us_zipcodes.csv")
# Problem: So erwartet er die Datei nicht in der hdfs-Root, sondern unter /user/alfa
# Verzeichnispfad
output_path = "/bigdata/temp_output"

# aufgeteilten DataFrame inj einer Datei schreiben
'''
Wir verwenden coalesce(1), um sicherzustellen, 
dass der DataFrame in einer einzigen Partition geschrieben wird, 
was in einer einzelnen Datei resultiert. 
'''
Sdf.coalesce(1).write.mode("overwrite").csv(output_path, header=True)

# HDFS-Client erstellen
hdfs_url        = 'http://172.26.195.52:9870'
ubuntu_Benutzer = 'alfa'
client = InsecureClient(hdfs_url, user=ubuntu_Benutzer)

# Dateien im HDFS-Verzeichnis auflisten
files = client.list(output_path)
csv_file = [file for file in files if file.endswith(".csv")][0]

# Quell- und Zielpfad für die umbenannte Datei
source_path = f"{output_path}/{csv_file}"
target_path = "/bigdata/test1234.csv"

# Datei umbenennen
client.rename(source_path, target_path)

print(f"Datei wurde unter {target_path} gespeichert")

spark.stop()
